<a href="https://colab.research.google.com/github/ssilb4/colab/blob/master/dacon_TabNet_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip uninstall -y typing # this should avoid  AttributeError: type object 'Callable' has no attribute '_abc_registry'

!pip install  "git+https://github.com/dreamquark-ai/tabnet.git@develop#egg=pytorch_tabnet" --upgrade

  Cloning https://github.com/dreamquark-ai/tabnet.git (to revision develop) to /tmp/pip-install-9f1rg8bk/pytorch-tabnet
  Running command git clone -q https://github.com/dreamquark-ai/tabnet.git /tmp/pip-install-9f1rg8bk/pytorch-tabnet
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pytorch-tabnet: filename=pytorch_tabnet-3.1.1-cp37-none-any.whl size=39236 sha256=4130c9e3cd04d97c2e0d980c478e70f5e1cd5d2937b3e546a6f7517a46bbada2
  Stored in directory: /tmp/pip-ephem-wheel-cache-dy27fsnm/wheels/a5/fe/e1/d7be493728d1fb7284583f5449d4cad80139ee994ef420f522
Successfully built pytorch-tabnet
  Found existing installation: pytorch-tabnet 3.1.1
    Uninstalling pytorch-tabnet-3.1.1:
      Successfully uninstalled pytorch-tabnet-3.1.1


In [3]:
from pytorch_tabnet.multitask import TabNetMultiTaskClassifier

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, log_loss

import pandas as pd
import numpy as np
np.random.seed(0)

from tqdm.notebook import tqdm

import os

from matplotlib import pyplot as plt
%matplotlib inline

In [4]:
path = '/content/drive/MyDrive/dacon/data/'

In [5]:
train = pd.read_csv(path+'train.csv')
train = train.drop(['index'], axis=1)
train.fillna('NAN', inplace=True) 


test = pd.read_csv(path+'test.csv')
test = test.drop(['index'], axis=1)
test.fillna('NAN', inplace=True)

submission = pd.read_csv(path+'sample_submission.csv')

In [6]:
data=pd.concat([train, test], axis=0)
data.shape

(36457, 19)

In [7]:
# 나이 변환
def days_to_age(x):
    return (x*-1)/365
data['DAYS_BIRTH'] = data['DAYS_BIRTH'].apply(days_to_age)

In [8]:
# 마이너스 값 변환
def minus(x):
    return x * -1
data['begin_month'] = data['begin_month'].apply(minus)

In [9]:
data.loc[(data.DAYS_EMPLOYED)>=0,'DAYS_EMPLOYED'] = 0

In [10]:
#data['DAYS_EMPLOYED'] = data['DAYS_EMPLOYED'].apply(days_to_age)

In [11]:
#data.loc[data['child_num'] >= 2,'child_num']=2

In [12]:
'''
data['income_total'] = data['income_total'].astype(object)
data['income_total'] = data['income_total']/10000 
print(data['income_total'].value_counts(bins=10,sort=False))
data['income_total'].plot(kind='hist',bins=50,density=True)
'''

"\ndata['income_total'] = data['income_total'].astype(object)\ndata['income_total'] = data['income_total']/10000 \nprint(data['income_total'].value_counts(bins=10,sort=False))\ndata['income_total'].plot(kind='hist',bins=50,density=True)\n"

In [13]:
'''
count, bin_dividers =np.histogram(data['income_total'], bins=7)
bin_names=[int(i) for i in range(7) ]
data['income_total']=pd.cut(x=data['income_total'], bins=bin_dividers, labels=bin_names, include_lowest=True)
'''

"\ncount, bin_dividers =np.histogram(data['income_total'], bins=7)\nbin_names=[int(i) for i in range(7) ]\ndata['income_total']=pd.cut(x=data['income_total'], bins=bin_dividers, labels=bin_names, include_lowest=True)\n"

In [14]:
print(data[data['income_type'] == 'Pensioner'].info())
data[data['income_type'] == 'Pensioner']['occyp_type'].unique()

print(data[(data['income_type'] == 'Pensioner') & (data['occyp_type'].isnull())])

data.loc[(data['income_type'] == 'Pensioner') & (data['occyp_type'].isnull()), 'occyp_type'] = 'no worker'

data.isnull().sum()

data[data['occyp_type'].isnull()]['income_type'].unique()

data['occyp_type'].unique()

print(data[data['occyp_type'] == 'Laborers']['income_type'].unique())
print(data[data['occyp_type'] == 'Low-skill Laborers']['income_type'].unique())
print(data[data['occyp_type'] == 'Managers']['income_type'].unique())

print(data[data['income_type'] == 'Commercial associate']['occyp_type'].describe())
data.loc[data['income_type'] == 'Commercial associate', 'occyp_type'] = 'Laborers'

print(data[data['income_type'] == 'Working']['occyp_type'].describe())
data.loc[data['income_type'] == 'Working', 'occyp_type'] = 'Laborers'

print(data[data['income_type'] == 'State servant']['occyp_type'].describe())
data.loc[data['income_type'] == 'State servant', 'occyp_type'] = 'Core staff'

print(data[data['income_type'] == 'Student']['occyp_type'].describe())
data.loc[data['income_type'] == 'Student', 'occyp_type'] = 'no worker'

data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6152 entries, 14 to 9994
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gender         6152 non-null   object 
 1   car            6152 non-null   object 
 2   reality        6152 non-null   object 
 3   child_num      6152 non-null   int64  
 4   income_total   6152 non-null   float64
 5   income_type    6152 non-null   object 
 6   edu_type       6152 non-null   object 
 7   family_type    6152 non-null   object 
 8   house_type     6152 non-null   object 
 9   DAYS_BIRTH     6152 non-null   float64
 10  DAYS_EMPLOYED  6152 non-null   int64  
 11  FLAG_MOBIL     6152 non-null   int64  
 12  work_phone     6152 non-null   int64  
 13  phone          6152 non-null   int64  
 14  email          6152 non-null   int64  
 15  occyp_type     6152 non-null   object 
 16  family_size    6152 non-null   float64
 17  begin_month    6152 non-null   float64
 18  credit 

gender               0
car                  0
reality              0
child_num            0
income_total         0
income_type          0
edu_type             0
family_type          0
house_type           0
DAYS_BIRTH           0
DAYS_EMPLOYED        0
FLAG_MOBIL           0
work_phone           0
phone                0
email                0
occyp_type           0
family_size          0
begin_month          0
credit           10000
dtype: int64

In [15]:
data['gender'] = data['gender'].replace(['F','M'],[0,1])
print('gender :')
print(data['gender'].value_counts())
print('--------------')

print('Having a car or not : ')
data['car'] = data['car'].replace(['N','Y'],[0,1])
print(data['car'].value_counts())
print('--------------')

print('Having house reality or not: ')
data['reality'] = data['reality'].replace(['N','Y'],[0,1])
print(data['reality'].value_counts())
print('--------------')
      
print('Having a phone or not: ')
print(data['phone'].value_counts())
print('--------------')
      

print('Having a email or not: ')
print(data['email'].value_counts())
print('--------------')
      

print('Having a work phone or not: ')
print(data['work_phone'].value_counts())
print('--------------')

gender :
0    24430
1    12027
Name: gender, dtype: int64
--------------
Having a car or not : 
0    22614
1    13843
Name: car, dtype: int64
--------------
Having house reality or not: 
1    24506
0    11951
Name: reality, dtype: int64
--------------
Having a phone or not: 
0    25709
1    10748
Name: phone, dtype: int64
--------------
Having a email or not: 
0    33186
1     3271
Name: email, dtype: int64
--------------
Having a work phone or not: 
0    28235
1     8222
Name: work_phone, dtype: int64
--------------


In [16]:
from sklearn import preprocessing
label_encoder=preprocessing.LabelEncoder()
data['income_type']=label_encoder.fit_transform(data['income_type'])
data['edu_type']=label_encoder.fit_transform(data['edu_type'])
data['family_type']=label_encoder.fit_transform(data['family_type'])
data['house_type']=label_encoder.fit_transform(data['house_type'])
data['income_total']=label_encoder.fit_transform(data['income_total'])
data['occyp_type']=label_encoder.fit_transform(data['occyp_type'])

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36457 entries, 0 to 9999
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gender         36457 non-null  int64  
 1   car            36457 non-null  int64  
 2   reality        36457 non-null  int64  
 3   child_num      36457 non-null  int64  
 4   income_total   36457 non-null  int64  
 5   income_type    36457 non-null  int64  
 6   edu_type       36457 non-null  int64  
 7   family_type    36457 non-null  int64  
 8   house_type     36457 non-null  int64  
 9   DAYS_BIRTH     36457 non-null  float64
 10  DAYS_EMPLOYED  36457 non-null  int64  
 11  FLAG_MOBIL     36457 non-null  int64  
 12  work_phone     36457 non-null  int64  
 13  phone          36457 non-null  int64  
 14  email          36457 non-null  int64  
 15  occyp_type     36457 non-null  int64  
 16  family_size    36457 non-null  float64
 17  begin_month    36457 non-null  float64
 18  credit 

In [43]:
train=data[:len(data)-10000]
test=data[len(data)-10000:]

In [44]:
train_x=train.drop('credit', axis=1)
train_y=train[['credit']]
test_x=test.drop('credit', axis=1)

In [45]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import OneSidedSelection

train_x, train_y = OneSidedSelection(random_state=1).fit_sample(train_x, train_y)
train_x, train_y = SMOTE(random_state=1).fit_sample(train_x, train_y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=F

In [46]:
train.columns.drop('credit')

Index(['gender', 'car', 'reality', 'child_num', 'income_total', 'income_type',
       'edu_type', 'family_type', 'house_type', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'FLAG_MOBIL', 'work_phone', 'phone', 'email', 'occyp_type',
       'family_size', 'begin_month'],
      dtype='object')

In [47]:
train_x = pd.DataFrame(train_x, columns = train.columns.drop('credit'))
train_y = pd.DataFrame(train_y, columns = ['credit'])

In [48]:
np.random.seed(42)
if "Set" not in train_x.columns:
    train_x["Set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(train_x.shape[0],))

train_indices = train_x[train_x.Set=="train"].index
valid_indices = train_x[train_x.Set=="valid"].index

In [52]:
train_x

0        2213
1        2213
2        2213
3        2213
4        2213
         ... 
44995    2213
44996       1
44997    1185
44998    2213
44999    2213
Name: occyp_type, Length: 45000, dtype: int64

In [53]:
test_x

0       7
1       1
2       3
3       3
4       1
       ..
9995    3
9996    3
9997    3
9998    3
9999    3
Name: occyp_type, Length: 10000, dtype: int64

In [51]:

nunique = train.nunique()
types = train.dtypes

categorical_columns = []
categorical_dims =  {}
for col in tqdm(train_x.columns.drop('Set')):
    if types[col] == 'object' or nunique[col] < 200:
        print(col, train_x[col].nunique())
        l_enc = LabelEncoder()
        train_x[col] = train_x[col].fillna("NaN")
        train_x[col] = l_enc.fit_transform(train_x[col].values)
        try:
            test_x[col] = test_x[col].fillna("NaN")
            test_x[col] = l_enc.transform(test_x[col].values)
        except:
            print(f"Column {col} does not exist in test set")
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        training_mean = train_x.loc[train_indices, col].mean()
        train.fillna(training_mean, inplace=True)
        test.fillna(training_mean, inplace=True)


gender 6585
car 6803
reality 7022
child_num 6859
income_type 6568
edu_type 7054
family_type 8196
house_type 3271
FLAG_MOBIL 1
work_phone 5123
phone 6798
email 2489
occyp_type 2228
Column occyp_type does not exist in test set
family_size 9911
begin_month 20655



/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [58]:
print(train_x)
print(test_x)

       gender   car  reality  ...  family_size  begin_month    Set
0           0     0     7021  ...         4065        16349  train
1        6584     0     7021  ...         4065        20060  valid
2           0  6802     7021  ...         8226        20654  train
3        6584  6802     7021  ...         9682        20654  train
4           0     0        0  ...         4065        11274  train
...       ...   ...      ...  ...          ...          ...    ...
44995       0     0        0  ...         4065        11641  train
44996       0     0        0  ...         9682        18790  train
44997    2992  3076     7021  ...         1826        17048  train
44998    6584  6802     7021  ...         9306         4116  train
44999       0  4978     5110  ...         8594          312  valid

[45000 rows x 19 columns]
      gender   car  reality  ...  occyp_type  family_size  begin_month
0       6584  6802        0  ...           7         4065        20654
1          0     0     7021

In [59]:
features = [ col for col in train_x.columns.drop('Set')] 

cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]

cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

In [60]:
X_train = train_x[features].values[train_indices]
y_train = train_y.values[train_indices]

X_valid = train_x[features].values[valid_indices]
y_valid = train_y.values[valid_indices]

X_test = test_x[features].values

In [61]:
clf = TabNetMultiTaskClassifier(n_steps=1,
                                cat_idxs=cat_idxs,
                                cat_dims=cat_dims,
                                cat_emb_dim=1,
                                optimizer_fn=torch.optim.Adam,
                                optimizer_params=dict(lr=2e-2),
                                scheduler_params={"step_size":50,
                                                  "gamma":0.9},
                                scheduler_fn=torch.optim.lr_scheduler.StepLR,
                                mask_type='entmax', 
                                lambda_sparse=0, 
                       
                      )

Device used : cuda


In [62]:
torch.backends.cudnn.benchmark = True

In [63]:
max_epochs = 1000
clf.fit(
    X_train=X_train, y_train=y_train,
    max_epochs=max_epochs ,
    patience=50, # please be patient ^^
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=1,
    drop_last=False,
)

No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 1.02497 |  0:00:01s
epoch 1  | loss: 0.91127 |  0:00:02s
epoch 2  | loss: 0.83778 |  0:00:03s
epoch 3  | loss: 0.77551 |  0:00:05s
epoch 4  | loss: 0.75016 |  0:00:06s
epoch 5  | loss: 0.72322 |  0:00:07s
epoch 6  | loss: 0.68825 |  0:00:08s
epoch 7  | loss: 0.6513  |  0:00:10s
epoch 8  | loss: 0.60269 |  0:00:11s
epoch 9  | loss: 0.57221 |  0:00:12s
epoch 10 | loss: 0.54832 |  0:00:14s
epoch 11 | loss: 0.53182 |  0:00:15s
epoch 12 | loss: 0.51854 |  0:00:16s
epoch 13 | loss: 0.50676 |  0:00:17s
epoch 14 | loss: 0.49948 |  0:00:19s
epoch 15 | loss: 0.49353 |  0:00:20s
epoch 16 | loss: 0.48422 |  0:00:21s
epoch 17 | loss: 0.48485 |  0:00:22s
epoch 18 | loss: 0.481   |  0:00:24s
epoch 19 | loss: 0.49348 |  0:00:25s
epoch 20 | loss: 0.48535 |  0:00:26s
epoch 21 | loss: 0.48094 |  0:00:27s
epoch 22 | loss: 0.47331 |  0:00:29s
epoch 23 | loss: 0.46757 |  0:00:30s
epoch 24 | loss: 0.4662  |  0:00:31s
ep

In [64]:
preds_valid = clf.predict_proba(X_valid) 

In [65]:
preds = clf.predict_proba(X_test)

In [66]:
clf.feature_importances_

array([0.1195931 , 0.00524409, 0.        , 0.        , 0.00901577,
       0.30460626, 0.22244109, 0.05156417, 0.        , 0.01572321,
       0.00376633, 0.00475578, 0.0548896 , 0.06220899, 0.        ,
       0.        , 0.04897032, 0.0972213 ])

In [67]:
i, j, k = [], [], []
for row in range(10000):
  i.append(preds[0][row][0])
  j.append(preds[0][row][1])
  k.append(preds[0][row][2])

len(i), len(j), len(k)

(10000, 10000, 10000)

In [68]:
submission['0'] = i
submission['1'] = j
submission['2'] = k
submission.head(10)

,index,0,1,2
0,26457,0.066923,0.105767,8.273093e-01
1,26458,0.001817,0.998183,6.506126e-08
2,26459,0.042324,0.685748,2.719281e-01
3,26460,0.117702,0.352815,5.294828e-01
4,26461,0.000117,0.999864,1.967510e-05
5,26462,0.000114,0.999882,3.608802e-06
6,26463,0.393776,0.594828,1.139584e-02
7,26464,0.012048,0.799113,1.888388e-01
8,26465,0.001397,0.998406,1.972991e-04
9,26466,0.039021,0.893072,6.790664e-02


In [69]:
submission.to_csv(path+"submisison_tabnet.csv",index=False)

In [70]:
prediction = pd.read_csv(path+'submisison_tabnet.csv')

In [71]:
prediction

,index,0,1,2
0,26457,0.066923,0.105767,8.273093e-01
1,26458,0.001817,0.998183,6.506126e-08
2,26459,0.042324,0.685748,2.719281e-01
3,26460,0.117702,0.352815,5.294828e-01
4,26461,0.000117,0.999864,1.967510e-05
...,...,...,...,...
9995,36452,0.032561,0.967439,7.561642e-07
9996,36453,0.004640,0.995335,2.428589e-05
9997,36454,0.003928,0.961027,3.504479e-02
9998,36455,0.983485,0.016515,5.658972e-14
